In [1]:
import pandas as pd
df = pd.read_csv('/home/xiaoguzai/数据/kaggle有毒评论对比/ruddit/ruddit_with_text.csv')
df.head()

,post_id,comment_id,txt,url,offensiveness_score
0,42g75o,cza1q49,> The difference in average earnings between m...,https://www.reddit.com/r/changemyview/comments...,-0.083
1,42g75o,cza1wdh,"The myth is that the ""gap"" is entirely based o...",https://www.reddit.com/r/changemyview/comments...,-0.022
2,42g75o,cza23qx,[deleted],https://www.reddit.com/r/changemyview/comments...,0.167
3,42g75o,cza2bw8,The assertion is that women get paid less for ...,https://www.reddit.com/r/changemyview/comments...,-0.146
4,42g75o,cza2iji,You said in the OP that's not what they're mea...,https://www.reddit.com/r/changemyview/comments...,-0.083


In [2]:
import re
import sys
import unicodedata

from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import SnowballStemmer
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse

import emoji

CUSTOM_TABLE = str.maketrans(
    {
        "\xad": None,
        "\x7f": None,
        "\ufeff": None,
        "\u200b": None,
        "\u200e": None,
        "\u202a": None,
        "\u202c": None,
        "‘": "'",
        "’": "'",
        "`": "'",
        "“": '"',
        "”": '"',
        "«": '"',
        "»": '"',
        "ɢ": "G",
        "ɪ": "I",
        "ɴ": "N",
        "ʀ": "R",
        "ʏ": "Y",
        "ʙ": "B",
        "ʜ": "H",
        "ʟ": "L",
        "ғ": "F",
        "ᴀ": "A",
        "ᴄ": "C",
        "ᴅ": "D",
        "ᴇ": "E",
        "ᴊ": "J",
        "ᴋ": "K",
        "ᴍ": "M",
        "Μ": "M",
        "ᴏ": "O",
        "ᴘ": "P",
        "ᴛ": "T",
        "ᴜ": "U",
        "ᴡ": "W",
        "ᴠ": "V",
        "ĸ": "K",
        "в": "B",
        "м": "M",
        "н": "H",
        "т": "T",
        "ѕ": "S",
        "—": "-",
        "–": "-",
    }
)

WORDS_REPLACER = [
    ("sh*t", "shit"),
    ("s**t", "shit"),
    ("f*ck", "fuck"),
    ("fu*k", "fuck"),
    ("f**k", "fuck"),
    ("f*****g", "fucking"),
    ("f***ing", "fucking"),
    ("f**king", "fucking"),
    ("p*ssy", "pussy"),
    ("p***y", "pussy"),
    ("pu**y", "pussy"),
    ("p*ss", "piss"),
    ("b*tch", "bitch"),
    ("bit*h", "bitch"),
    ("h*ll", "hell"),
    ("h**l", "hell"),
    ("cr*p", "crap"),
    ("d*mn", "damn"),
    ("stu*pid", "stupid"),
    ("st*pid", "stupid"),
    ("n*gger", "nigger"),
    ("n***ga", "nigger"),
    ("f*ggot", "faggot"),
    ("scr*w", "screw"),
    ("pr*ck", "prick"),
    ("g*d", "god"),
    ("s*x", "sex"),
    ("a*s", "ass"),
    ("a**hole", "asshole"),
    ("a***ole", "asshole"),
    ("a**", "ass"),
]

REGEX_REPLACER = [
    (re.compile(pat.replace("*", "\*"), flags=re.IGNORECASE), repl)
    for pat, repl in WORDS_REPLACER
]

RE_SPACE = re.compile(r"\s")
RE_MULTI_SPACE = re.compile(r"\s+")

NMS_TABLE = dict.fromkeys(
    i for i in range(sys.maxunicode + 1) if unicodedata.category(chr(i)) == "Mn"
)

HEBREW_TABLE = {i: "א" for i in range(0x0590, 0x05FF)}
ARABIC_TABLE = {i: "ا" for i in range(0x0600, 0x06FF)}
CHINESE_TABLE = {i: "是" for i in range(0x4E00, 0x9FFF)}
KANJI_TABLE = {i: "ッ" for i in range(0x2E80, 0x2FD5)}
HIRAGANA_TABLE = {i: "ッ" for i in range(0x3041, 0x3096)}
KATAKANA_TABLE = {i: "ッ" for i in range(0x30A0, 0x30FF)}

TABLE = dict()
TABLE.update(CUSTOM_TABLE)
TABLE.update(NMS_TABLE)
# Non-english languages
TABLE.update(CHINESE_TABLE)
TABLE.update(HEBREW_TABLE)
TABLE.update(ARABIC_TABLE)
TABLE.update(HIRAGANA_TABLE)
TABLE.update(KATAKANA_TABLE)
TABLE.update(KANJI_TABLE)


EMOJI_REGEXP = emoji.get_emoji_regexp()

UNICODE_EMOJI_MY = {
    k: f" EMJ {v.strip(':').replace('_', ' ')} "
    for k, v in emoji.UNICODE_EMOJI_ALIAS_ENGLISH.items()
}


def my_demojize(string: str) -> str:
    """
    Modified version of the demojize function from emoji
    """

    def replace(match):
        return UNICODE_EMOJI_MY.get(match.group(0), match.group(0))

    return re.sub("\ufe0f", "", EMOJI_REGEXP.sub(replace, string))


def fix_tokens(tokens):
    """
    Expects a list of lower-cased tokens from TweeterTokenizer
    """
    for token in tokens:
        if token == "gov't" or token == "govt":
            yield "government"
        elif token == "i'm":
            yield "i"
            yield "am"
        elif token.endswith("n't"):
            yield token[:-3]
            yield "not"
        elif token.endswith("'re"):
            yield token[:-3]
            yield "are"
        elif token.endswith("'ll"):
            yield token[:-3]
            yield "will"
        elif token.endswith("'ve"):
            yield token[:-3]
            yield "have"
        elif token.endswith("'s"):
            yield token[:-2]
            yield "'s"
        else:
            yield token


def normalize(text: str) -> str:
    text = my_demojize(text)
    text = RE_SPACE.sub(" ", text)
    text = unicodedata.normalize("NFKD", text)
    text = text.translate(TABLE)
    text = RE_MULTI_SPACE.sub(" ", text).strip()

    for pattern, repl in REGEX_REPLACER:
        text = pattern.sub(repl, text)

    return text

def text_cleaning(text):
    '''
    Cleans text into a basic form for NLP. Operations include the following:-
    1. Remove special charecters like &, #, etc
    2. Removes extra spaces
    3. Removes embedded URL links
    4. Removes HTML tags
    5. Removes emojis
    
    text - Text piece to be cleaned.
    '''
    template = re.compile(r'https?://\S+|www\.\S+') #Removes website links
    text = template.sub(r'', text)
    
    soup = BeautifulSoup(text, 'lxml') #Removes HTML tags
    only_text = soup.get_text()
    text = only_text
    
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    
    text = re.sub(r"[^a-zA-Z\d]", " ", text) #Remove special Charecters
    text = re.sub(' +', ' ', text) #Remove Extra Spaces
    text = text.strip() # remove spaces at the beginning and at the end of string


    text = text.lower()
    # Clean some punctutations
    text = text.replace('\n', ' \n ')  
    # Add space between pattern
    text = re.sub(r'([a-zA-Z]+)([/!?.])([a-zA-Z]+)',r'\1 \2 \3', text)
    # Replace repeating puncts more than 3 times to length of 3
    text = re.sub(r'([*!?\'])\1\1{2,}',r'\1\1\1', text)    
    # Add space around repeating puncts
    text = re.sub(r'([*!?\']+)',r' \1 ', text)    
    # patterns with repeating characters 
    # naaaahhhhhhh to naaaahh
    text = re.sub(r'([a-zA-Z])\1{2,}\b',r'\1\1', text)
    # naaaahh to naaahh
    text = re.sub(r'([a-zA-Z])\1\1{2,}\B',r'\1\1\1', text)
    # multiple spaces to one
    text = re.sub(r'[ ]{2,}',' ', text).strip()   
    text = normalize(text)


    return text



In [3]:
from pandarallel import pandarallel
pandarallel.initialize(nb_workers=4)
df['txt'] = df['txt'].parallel_apply(normalize)
df['txt'] = df['txt'].parallel_apply(text_cleaning)

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [4]:
text_list = df['txt'].values.tolist()
print(len(text_list))

5838


In [5]:
df.head()

,post_id,comment_id,txt,url,offensiveness_score
0,42g75o,cza1q49,the difference in average earnings between men...,https://www.reddit.com/r/changemyview/comments...,-0.083
1,42g75o,cza1wdh,the myth is that the gap is entirely based on ...,https://www.reddit.com/r/changemyview/comments...,-0.022
2,42g75o,cza23qx,deleted,https://www.reddit.com/r/changemyview/comments...,0.167
3,42g75o,cza2bw8,the assertion is that women get paid less for ...,https://www.reddit.com/r/changemyview/comments...,-0.146
4,42g75o,cza2iji,you said in the op that s not what they re mea...,https://www.reddit.com/r/changemyview/comments...,-0.083


In [6]:
my_train_text = []
for currenttext in df['txt']:
    my_train_text.append(currenttext.split(' '))

In [7]:
from gensim.models import FastText
model = FastText(vector_size=4, min_count=4)  # instantiate
model.build_vocab(my_train_text)
model.train(my_train_text, epochs=10, total_examples=len(my_train_text))  # train

(1416643, 2099800)

In [8]:
print("单词user的词向量：", model.wv.get_vector('user'))
model.save('/home/xiaoguzai/模型/fasttext-linear/ruddit_result.bin', separately=[])

单词user的词向量： [ 0.8211108 -1.0423776  2.2726233  1.1154736]


In [9]:
model = FastText.load('/home/xiaoguzai/模型/fasttext-linear/ruddit_result.bin')

In [10]:
vec = TfidfVectorizer(min_df= 3, max_df=0.5, analyzer = 'char_wb', ngram_range = (3,5), max_features = 46000)
vec.fit(df['txt'])

TfidfVectorizer(analyzer='char_wb', max_df=0.5, max_features=46000, min_df=3,
                ngram_range=(3, 5))

In [11]:
from scipy.sparse import hstack
def splitter(text):
    tokens = []
    
    for word in text.split(' '):
        tokens.append(word)
    
    return tokens

def vectorizer(text):
    tokens = splitter(text)
    
    x1 = vec.transform([text]).toarray()
    x2 = np.mean(model.wv[tokens], axis = 0).reshape(1, -1)
    x = np.concatenate([x1, x2], axis = -1).astype(np.float16)
    del x1
    del x2 
    
    #return x[0] 
    return x[0]

In [12]:
import numpy as np
X_list = []
for currenttext in df.txt:
    X_list.append(vectorizer(currenttext))
    
labels = df['offensiveness_score'].values.tolist()

In [13]:
X_list[0]

array([ 0.    ,  0.    ,  0.    , ..., -0.4456,  0.9937,  2.58  ],
      dtype=float16)

In [14]:
labels[0]

-0.083

In [15]:
from sklearn.linear_model import Ridge
regressor1 = Ridge(random_state=42,alpha=0.8)
regressor1.fit(X_list,labels) 

Ridge(alpha=0.8, random_state=42)

In [16]:
#regressor = Ridge(random_state=42,alpha=0.8)
#regressor.fit(comments_tr,labels)

In [17]:
df_val = pd.read_csv('/home/xiaoguzai/数据/Kaggle Jigsaw Rate Severity of Toxic Comments/validation_true.csv')
df_val.head()

,Unnamed: 0,less_toxic,more_toxic
0,1,I just read what you wrote; neither of the tw...,\n\nwhy should people have to read crap posted...
1,2,"""\n\nI don't know which """"personal attacks"""" y...",\n\nwhy should people have to read crap posted...
2,5,"\nU POUR ADMIN, U UPDATE VANDAL COUNT, WHILE R...",Your relentless edit warring \n\nYour message...
3,9,\n\nFailure to follow the guidelines of WP:NO...,Against community and you don't know what devi...
4,10,\n\nGet a encyclopedia you nerds......(kanich...,"AND TO THE PARTY WHO FUCKS ME, FUCK U! I'LL BE..."


In [18]:
df_val['less_toxic'] = df_val['less_toxic'].parallel_apply(normalize)
df_val['less_toxic'] = df_val['less_toxic'].parallel_apply(text_cleaning)
df_val['more_toxic'] = df_val['more_toxic'].parallel_apply(normalize)
df_val['more_toxic'] = df_val['more_toxic'].parallel_apply(text_cleaning)

In [19]:
df_val_less_list = []
df_val_more_list = []
for current_text in df_val.less_toxic:
    df_val_less_list.append(vectorizer(current_text))
for current_text in df_val.more_toxic:
    df_val_more_list.append(vectorizer(current_text))

In [20]:
df_val_less_result = regressor1.predict(df_val_less_list)
df_val_more_result = regressor1.predict(df_val_more_list)

In [21]:
np.round((df_val_less_result < df_val_more_result).mean() * 100,2)

75.95